In [1]:
from transformers import BertTokenizer, AutoModelForMaskedLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset
import torch

In [2]:
dataset = load_dataset("ErikCikalleshi/new_york_times_news_1987_1995")

In [ ]:
ds = dataset['train']
df = ds.to_pandas()
for data in df:
    data['content'] = f'<year_{data['date']}> + '

,date,title,content
0,1987,BRELAND FINDS PRO STYLE WITH KNOCKOUT PUNCH,"LEAD: As an amateur, Mark Breland seemed the p..."
1,1994,"Foreman's Fistic Follies (Or, Just What Is You...",What was George Foreman trying to tell Jim Lam...
2,1989,Real Estate,"LEAD: IN the last few years, huge developments..."
3,1995,Residential Resales,"In the following reports, the ""listed at"" pric..."
4,1995,Capitals Top Isles to Boost Playoff Bid,The Washington Capitals held off the pesky Isl...
...,...,...,...
581846,1992,THE 1992 CAMPAIGN: The Democrats; Clinton Read...,Choosing to confront an issue he had declared ...
581847,1987,"IN A 1938 FLIGHT TO SOVIET, LOVE KNEW NO FRONT...","LEAD: Almost 50 years ago, a lovelorn young Br..."
581848,1988,Large Sale Of Florida Real Estate,"LEAD: Shearson Lehman Hutton Inc., Triangle In..."
581849,1988,Possessing Dukakis,LEAD: WE CAN ALL agree: the 1988 Democratic Na...


In [4]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
tokenizer.add_special_tokens()
tokenized_dataset = dataset.map(lambda x: tokenizer(x["text"], truncation=True, padding=True), batched=True)




tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

TypeError: PreTrainedTokenizerBase.add_special_tokens() missing 1 required positional argument: 'special_tokens_dict'

In [ ]:
# Data collator for Masked Language Modeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

# Load pre-trained model
model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")

In [ ]:
training_args = TrainingArguments(
    output_dir="./domain_pretrained_model",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    save_steps=10_000,
    save_total_limit=2,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    data_collator=data_collator,
)

# Pretrain the model
trainer.train()